In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("C:/Users/isabella.bismanns/Downloads/TechLabs/Projekt/TechLab_DSProjectTeam1/loan_data.csv", delimiter=";")
df.info()


df["CoapplicantIncome"]=pd.to_numeric(df["CoapplicantIncome"], errors="coerce") 
#does not use the two big values from LP001915 and LP002369

df["CoapplicantIncome"].mean()
df["CoapplicantIncome"].median()

df["TotalIncome"]=df["ApplicantIncome"]+df["CoapplicantIncome"]

df.describe()

df.boxplot(column="TotalIncome")


ModuleNotFoundError: No module named 'matplotlib'